### ⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯
# Laboratory 1 Solution
### Cooperating with: Riccardo Vaccari (s348856)
### ⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯

## Import

In [1]:
import numpy as np
from tqdm.auto import tqdm

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Setup

In [12]:
NUM_SAMPLES = 10  # number of candidates solution
MAX_STEPS = 10000

In [9]:
# Problem 1 setup
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS,NUM_DIMENSIONS))


In [15]:
# Problem 2 setup
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(1000 * 2, 1000 * NUM_ITEMS // NUM_KNAPSACKS,  size=(NUM_KNAPSACKS,NUM_DIMENSIONS))

In [35]:
# Problem 3 setup
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(1000 * 10, 1000 * 2 * NUM_ITEMS // NUM_KNAPSACKS,  size=(NUM_KNAPSACKS,NUM_DIMENSIONS))

## Base Hill Climbing

In [16]:
# start with empty solution (no items assigned)
solution = np.zeros((NUM_KNAPSACKS, NUM_ITEMS), dtype=bool)

# fitness function
def value(sol):
    return VALUES[np.any(sol, axis=0)].sum()

# validity check
def is_valid(sol):
    # each item in at most one knapsack
    if np.any(sol.sum(axis=0) > 1):
        return False

    # each knapsack respects its constraints
    return np.all([WEIGHTS[sol[k]].sum(axis=0) <= CONSTRAINTS[k] for k in range(NUM_KNAPSACKS)])

# tweak: flip one random item in one random knapsack
def tweak(sol):
    new_solution = solution.copy()
    k = rng.integers(0, NUM_KNAPSACKS)
    i = rng.integers(0, NUM_ITEMS)
    new_solution[k, i] = not new_solution[k, i]
    return new_solution

# --- Hill climbing loop ---
for _ in tqdm(range(MAX_STEPS)):
    new_solution = tweak(solution)

    if not is_valid(new_solution):
        continue
    
    # accept if equal or better to avoid remaining stuck
    if value(new_solution) >= value(solution):
        solution = new_solution

# Results 
total_value = value(solution)
values_per_knapsack = [VALUES[solution[k]].sum() for k in range(NUM_KNAPSACKS)]

print("Total value:", total_value)
for k, v in enumerate(values_per_knapsack):
    print(f"Zaino {k+1} value: {v}")

for k in range(NUM_KNAPSACKS):
    total_weights = WEIGHTS[solution[k]].sum(axis=0)
    print(f"Zaino {k+1} weights: {total_weights}, constraints: {CONSTRAINTS[k]}")

100%|██████████| 10000/10000 [00:00<00:00, 67039.79it/s]

Total value: 34523
Zaino 1 value: 3845
Zaino 2 value: 2882
Zaino 3 value: 3118
Zaino 4 value: 4271
Zaino 5 value: 4647
Zaino 6 value: 4532
Zaino 7 value: 2141
Zaino 8 value: 1458
Zaino 9 value: 3101
Zaino 10 value: 4528
Zaino 1 weights: [2305 4086 3681 2635 3690 2909 3130 1839 2140 2222], constraints: [5837 5754 9351 3205 4450 3447 3952 9256 9525 2357]
Zaino 2 weights: [2192 3687 3459 2212 2891 1753 2772 3000 3020 1803], constraints: [7056 3862 3540 4336 3555 5921 9964 6691 7785 5946]
Zaino 3 weights: [2075 1829 2851 3317 3711 2685 2048 2266 2829 1189], constraints: [2111 2672 4150 3949 8724 8748 8832 7100 6449 7193]
Zaino 4 weights: [3011 3741 2303 3626 3470 2417 3702 3355 3056 3712], constraints: [5254 7361 8355 8103 6910 2464 7799 4932 7949 6316]
Zaino 5 weights: [5579 4471 4875 3896 4635 4920 5766 3253 6756 4067], constraints: [9049 4707 9451 8755 4648 5860 6250 8149 7513 8816]
Zaino 6 weights: [2958 3150 2351 3405 4271 4095 2932 4876 3716 3609], constraints: [4958 6038 5446 9276 4

## Hill Climbing (with Tabu Search)

In [17]:
# start with empty solution (no items assigned)
solution = np.zeros((NUM_KNAPSACKS, NUM_ITEMS), dtype=bool)

# fitness function
def value(sol):
    return VALUES[np.any(sol, axis=0)].sum()

# validity check
def is_valid(sol):
    # each item in at most one knapsack
    if np.any(sol.sum(axis=0) > 1):
        return False
    # each knapsack respects its constraints
    return np.all([WEIGHTS[sol[k]].sum(axis=0) <= CONSTRAINTS[k] for k in range(NUM_KNAPSACKS)])

# tweak: flip one random item in one random knapsack
def tweak(sol):
    new_solution = sol.copy()
    k = rng.integers(0, NUM_KNAPSACKS)
    i = rng.integers(0, NUM_ITEMS)
    new_solution[k, i] = not new_solution[k, i]
    return new_solution

# --- Hill Climbing loop ---

tabu_list = set()

for _ in tqdm(range(MAX_STEPS)):
    possible_solutions = [tweak(solution) for _ in range(NUM_SAMPLES)]
    candidates = [
        sol for sol in possible_solutions 
        if is_valid(sol) and sol.flatten().tobytes() not in tabu_list
    ]

    if not candidates:
        continue

    new_solution = max(candidates, key=value)

    # accept if equal or better to avoid remaining stuck
    if value(new_solution) >= value(solution):
        tabu_list.add(solution.flatten().tobytes())
        solution = new_solution

# Results 
total_value = value(solution)
values_per_knapsack = [VALUES[solution[k]].sum() for k in range(NUM_KNAPSACKS)]

print("Total value:", total_value)
for k, v in enumerate(values_per_knapsack):
    print(f"Knapsack {k+1} value: {v}")

for k in range(NUM_KNAPSACKS):
    total_weights = WEIGHTS[solution[k]].sum(axis=0)
    print(f"Knapsack {k+1} weights: {total_weights}, constraints: {CONSTRAINTS[k]}")

100%|██████████| 10000/10000 [00:01<00:00, 8314.40it/s]

Total value: 42024
Knapsack 1 value: 4544
Knapsack 2 value: 4062
Knapsack 3 value: 4422
Knapsack 4 value: 3483
Knapsack 5 value: 5588
Knapsack 6 value: 5484
Knapsack 7 value: 4400
Knapsack 8 value: 2491
Knapsack 9 value: 2918
Knapsack 10 value: 4632
Knapsack 1 weights: [2731 5271 4728 3150 2965 2719 2936 2799 3097 2068], constraints: [5837 5754 9351 3205 4450 3447 3952 9256 9525 2357]
Knapsack 2 weights: [2940 3670 2454 3445 3169 3561 3404 2766 3394 3231], constraints: [7056 3862 3540 4336 3555 5921 9964 6691 7785 5946]
Knapsack 3 weights: [1946 2629 3985 3768 3593 2783 4167 2648 3549 3301], constraints: [2111 2672 4150 3949 8724 8748 8832 7100 6449 7193]
Knapsack 4 weights: [1723 2075 3411 2868 2520 2446 2794 2004 1250 3176], constraints: [5254 7361 8355 8103 6910 2464 7799 4932 7949 6316]
Knapsack 5 weights: [5376 4598 5554 4776 4634 5827 5263 4323 5061 5106], constraints: [9049 4707 9451 8755 4648 5860 6250 8149 7513 8816]
Knapsack 6 weights: [4882 3187 4607 4720 4128 5701 2932 3264